<a href="https://colab.research.google.com/github/HasiniKola/SlientSentry_FallDetection/blob/main/EPICS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip MobiFall_Dataset_v2.0.zip

Archive:  MobiFall_Dataset_v2.0.zip
  inflating: MobiFall_Dataset_v2.0/Readme.txt  
  inflating: MobiFall_Dataset_v2.0/sub1/FALLS/BSC/BSC_acc_1_1.txt  
  inflating: MobiFall_Dataset_v2.0/sub1/FALLS/BSC/BSC_acc_1_2.txt  
  inflating: MobiFall_Dataset_v2.0/sub1/FALLS/BSC/BSC_acc_1_3.txt  
  inflating: MobiFall_Dataset_v2.0/sub1/FALLS/BSC/BSC_gyro_1_1.txt  
  inflating: MobiFall_Dataset_v2.0/sub1/FALLS/BSC/BSC_gyro_1_2.txt  
  inflating: MobiFall_Dataset_v2.0/sub1/FALLS/BSC/BSC_gyro_1_3.txt  
  inflating: MobiFall_Dataset_v2.0/sub1/FALLS/BSC/BSC_ori_1_1.txt  
  inflating: MobiFall_Dataset_v2.0/sub1/FALLS/BSC/BSC_ori_1_2.txt  
  inflating: MobiFall_Dataset_v2.0/sub1/FALLS/BSC/BSC_ori_1_3.txt  
  inflating: MobiFall_Dataset_v2.0/sub1/FALLS/FKL/FKL_acc_1_1.txt  
  inflating: MobiFall_Dataset_v2.0/sub1/FALLS/FKL/FKL_acc_1_2.txt  
  inflating: MobiFall_Dataset_v2.0/sub1/FALLS/FKL/FKL_acc_1_3.txt  
  inflating: MobiFall_Dataset_v2.0/sub1/FALLS/FKL/FKL_gyro_1_1.txt  
  inflating: MobiFall_Datase

In [2]:
!ls


MobiFall_Dataset_v2.0  MobiFall_Dataset_v2.0.zip  sample_data


In [3]:
import os
import pandas as pd

DATASET_ROOT = "MobiFall_Dataset_v2.0"
rows = []

for subject in os.listdir(DATASET_ROOT):
    subject_path = os.path.join(DATASET_ROOT, subject)
    if not os.path.isdir(subject_path):
        continue

    for activity in ["ADL", "FALLS"]:
        activity_path = os.path.join(subject_path, activity)
        if not os.path.exists(activity_path):
            continue

        label = 0 if activity == "ADL" else 1

        for subfolder in os.listdir(activity_path):
            subfolder_path = os.path.join(activity_path, subfolder)
            if not os.path.isdir(subfolder_path):
                continue

            for file in os.listdir(subfolder_path):
                # ✅ Use only accelerometer files
                if "_acc_" not in file:
                    continue

                file_path = os.path.join(subfolder_path, file)

                with open(file_path, "r") as f:
                    for line in f:
                        parts = line.strip().split(",")
                        if len(parts) < 4:
                            continue
                        try:
                            ax = float(parts[1])
                            ay = float(parts[2])
                            az = float(parts[3])
                            rows.append([ax, ay, az, label])
                        except:
                            continue

df = pd.DataFrame(rows, columns=["ax", "ay", "az", "label"])
df.to_csv("mobifall_clean.csv", index=False)

print("✅ Dataset extraction completed")
print("Total rows:", len(df))


✅ Dataset extraction completed
Total rows: 1044765


# New Section

In [4]:
import pandas as pd

df = pd.read_csv("mobifall_clean.csv")

df.head()


,ax,ay,az,label
0,1.072602,9.749189,1.082179,1
1,1.110910,9.749189,0.756568,1
2,1.082179,9.749189,1.053449,1
3,1.072602,9.758766,1.206678,1
4,1.139640,9.749189,0.794875,1


In [5]:
print("Total samples:", len(df))
print(df["label"].value_counts())


Total samples: 1044765
label
0    803334
1    241431
Name: count, dtype: int64


In [6]:
fall_df = df[df["label"] == 1]
adl_df = df[df["label"] == 0]

adl_sampled = adl_df.sample(n=len(fall_df), random_state=42)

balanced_df = pd.concat([fall_df, adl_sampled])

balanced_df = balanced_df.sample(frac=1, random_state=42)  # shuffle

print(balanced_df["label"].value_counts())


label
0    241431
1    241431
Name: count, dtype: int64


In [8]:
balanced_df.to_csv("mobifall_balanced.csv", index=False)


STEP 3 PREPROCESSING AND ML TRAINING

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = balanced_df[['ax', 'ay', 'az']]
y = balanced_df['label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Training samples:", X_train_scaled.shape[0])
print("Testing samples:", X_test_scaled.shape[0])


Training samples: 338003
Testing samples: 144859


In [10]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train_scaled, y_train)

print("✅ Random Forest training completed")


✅ Random Forest training completed


In [11]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = rf_model.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.9120178932617234

Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.93      0.91     72430
           1       0.93      0.89      0.91     72429

    accuracy                           0.91    144859
   macro avg       0.91      0.91      0.91    144859
weighted avg       0.91      0.91      0.91    144859


Confusion Matrix:
 [[67708  4722]
 [ 8023 64406]]


In [12]:
import joblib

joblib.dump(rf_model, "rf_fall_model.pkl")
joblib.dump(scaler, "scaler.pkl")


['scaler.pkl']

Step 5.1: Load the saved model & scaler

In [13]:
import joblib
import numpy as np

# Load saved model and scaler
rf_model = joblib.load("rf_fall_model.pkl")
scaler = joblib.load("scaler.pkl")

print("✅ Model and scaler loaded successfully")


✅ Model and scaler loaded successfully


Step 5.2: Test prediction on a sample (manual test)

In [14]:
# Take one sample from balanced dataset
sample = balanced_df[['ax', 'ay', 'az']].iloc[0].values.reshape(1, -1)

sample_scaled = scaler.transform(sample)
prediction = rf_model.predict(sample_scaled)

print("Prediction (0=Normal, 1=Fall):", prediction[0])


Prediction (0=Normal, 1=Fall): 0


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


STEP 6: REAL-TIME INFERENCE LOGIC (CORE IMPLEMENTATION)
ESP32 → ax, ay, az
      → Python script
      → scaler
      → Random Forest
      → if Fall → Telegram alert

Step 6.1: Define real-time prediction function

In [15]:
def predict_fall(ax, ay, az):
    data = np.array([[ax, ay, az]])
    data_scaled = scaler.transform(data)
    prediction = rf_model.predict(data_scaled)[0]
    probability = rf_model.predict_proba(data_scaled)[0][1]
    return prediction, probability


In [16]:
# Simulated ESP32 values
ax, ay, az = -8.5, -11.2, 3.9

pred, prob = predict_fall(ax, ay, az)

print("Prediction:", "FALL" if pred == 1 else "NORMAL")
print("Fall probability:", round(prob, 2))


Prediction: FALL
Fall probability: 0.93


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


STEP 7: TELEGRAM ALERT INTEGRATION (FINAL SYSTEM)

In [17]:
import requests

BOT_TOKEN = "8449810896:AAHgDMYGF-rcqLoJSYR6IL6IENHUMraKJ8E"
CHAT_ID = "5793538368"

def send_telegram_alert(message):
    url = f"https://api.telegram.org/bot{BOT_TOKEN}/sendMessage"
    data = {"chat_id": CHAT_ID, "text": message}
    requests.post(url, data=data)


In [18]:
def handle_realtime_data(ax, ay, az):
    pred, prob = predict_fall(ax, ay, az)

    if pred == 1 and prob > 0.7:
        send_telegram_alert(
            f"🚨 FALL DETECTED\nax={ax}, ay={ay}, az={az}\nConfidence={prob:.2f}"
        )
        print("🚨 Alert sent")
    else:
        print("✅ Normal activity")


In [19]:
handle_realtime_data(-8.5, -11.2, 3.9)


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


🚨 Alert sent


In [20]:
from google.colab import files

files.download("rf_fall_model.pkl")
files.download("scaler.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>